In [2]:
import pandas as pd
from pathlib import Path

# Get the base directory = parent of the current notebook
BASE_DIR = Path.cwd().parent   

# build data path
data_path = BASE_DIR / "1. data" / "raw" / "acetp" / "extracted"

# Step 1: Load CSV
df_pl = pd.read_csv(data_path / "PL Data.csv")

# Step 2: Drop rows where PL_Year End is blank
df_pl = df_pl.dropna(subset=["PL_Year End"])
df_pl = df_pl[df_pl["PL_Year End"].astype(str).str.strip() != ""]
df_pl = df_pl.reset_index(drop=True)

# Step 3: Calculate OR
df_pl["OR"] = (
    df_pl["PL_Net Sales"].fillna(0) +
    df_pl["PL_      Foreign Exchange Gains"].fillna(0) +
    df_pl["PL_      Grants from Government"].fillna(0) +
    df_pl["PL_      Bad debts recovered"].fillna(0) +
    df_pl["PL_      Provision Written Back"].fillna(0) +
    df_pl["PL_      Export Incentives"].fillna(0)
)

# Step 4: Calculate OC
df_pl["OC"] = (
    df_pl["PL_Total Expenditure"].fillna(0) +
    df_pl["PL_Depreciation"].fillna(0) -
    df_pl["PL_Miscellaneous Expenses"].fillna(0) +
    df_pl["PL_      Others"].fillna(0) +
    df_pl["PL_      Bad debts/advances written off"].fillna(0) +
    df_pl["PL_      Provision for doubtful debts"].fillna(0) +
    df_pl["PL_      Loss on foreign exchange fluctuations"].fillna(0) +
    df_pl["PL_      Other Miscellaneous Expenses"].fillna(0)
)

# Step 5: Merge with Segment Mapping (only 'Segment' column)
df_seg = pd.read_csv(data_path / "Seg_Mapping.csv", usecols=["Accord Code", "Segment"])
df_pl = df_pl.merge(df_seg, on="Accord Code", how="left")
df_pl["Segment"] = df_pl["Segment"].fillna("Other")


# Step 5: Drop the source columns
cols_to_drop = [
    "Sr.No.",
    "PL_Net Sales",
    "PL_      Foreign Exchange Gains",
    "PL_      Grants from Government",
    "PL_      Bad debts recovered",
    "PL_      Provision Written Back",
    "PL_      Export Incentives",
    "PL_Total Expenditure",
    "PL_Depreciation",
    "PL_Miscellaneous Expenses",
    "PL_      Others",
    "PL_      Bad debts/advances written off",
    "PL_      Provision for doubtful debts",
    "PL_      Loss on foreign exchange fluctuations",
    "PL_      Other Miscellaneous Expenses",
    "CD_Sector",
    "CD_Business Description",
    "CD_Economic Activity(NIC)",
    "CD_Nic Code",
    "CD_Sub Industry"
]

df_pl = df_pl.drop(columns=cols_to_drop, errors="ignore")
df_pl.head()


,Accord Code,Company Name,CD_Industry1,CD_CIN Number,PL_Year End,PL_ Sales - Exports,PL_ Manufacturing Sales,PL_ Trading Sales,PL_ Services Sales,PL_Employee Cost,OR,OC,Segment
0,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202403,91.08,669.58,NaN,3.22,49.50,673.17,597.76,Other
1,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202303,86.34,595.42,NaN,2.78,44.13,598.95,540.67,Other
2,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202203,82.75,514.78,NaN,2.79,36.68,517.84,462.81,Other
3,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,202403,8.51,92.14,NaN,NaN,8.23,92.23,83.94,Other
4,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,202303,9.12,87.57,NaN,0.49,9.45,88.22,80.98,Other


In [3]:
import pandas as pd

# Step 1: Load CSV
df_bs = pd.read_csv(data_path / "BS Data.csv")

# Step 2: Drop rows where BS_Year End is blank
df_bs = df_bs.dropna(subset=["BS_Year End"])
df_bs = df_bs[df_bs["BS_Year End"].astype(str).str.strip() != ""]
df_bs = df_bs.reset_index(drop=True)

# Step 3: Calculate Gross_Intbls
df_bs["Gross_Intbls"] = (
    df_bs["BS_Goodwill"].fillna(0) +
    df_bs["BS_Technical know-how"].fillna(0) +
    df_bs["BS_Patents, trademarks and designs"].fillna(0) +
    df_bs["BS_Technology License  Fees"].fillna(0)
)

# Step 4: Drop unwanted columns
cols_to_drop = [
    "Sr.No.",
    "BS_Goodwill",
    "BS_Computer Software",
    "BS_Technical know-how",
    "BS_Patents, trademarks and designs",
    "BS_Technology License  Fees"
]

df_bs = df_bs.drop(columns=cols_to_drop, errors="ignore")
df_bs.head()


,Accord Code,Company Name,BS_Year End,BS_Sundry Debtors,Gross_Intbls
0,207635,20 Microns Ltd.,202403,112.55,0.0
1,207635,20 Microns Ltd.,202303,89.57,0.0
2,207635,20 Microns Ltd.,202203,95.45,0.0
3,224943,20 Microns Nano Minerals Ltd.,202403,10.39,0.0
4,224943,20 Microns Nano Minerals Ltd.,202303,9.52,0.0


In [4]:
import pandas as pd

# Step 1: Load CSV
df_rp = pd.read_csv(data_path / "RP Data.csv")

# Step 2: Drop rows where RPTS_Year End is blank
df_rp = df_rp.dropna(subset=["RPTS_Year End"])
df_rp = df_rp[df_rp["RPTS_Year End"].astype(str).str.strip() != ""]
df_rp = df_rp.reset_index(drop=True)

# Step 3: Calculate RP_Inc
income_fields = [
    "RPTS_Sales",
    "RPTS_Rent Income",
    "RPTS_Dividend Income",
    "RPTS_Interest Income",
    "RPTS_Forex Gain",
    "RPTS_Discount Income",
    "RPTS_Commission Income",
    "RPTS_Claims Received",
    "RPTS_Other Income",
    "RPTS_Profit on Sale of Assets",
    "RPTS_Profit on Sale of Investments",
    "RPTS_Provision Written Back",
    "RPTS_Reimbursement Income",
    "RPTS_Sale of Assets",
    "RPTS_Sale of Investments",
    "RPTS_Other Non Operating income"
]

df_rp["RP_Inc"] = df_rp[income_fields].fillna(0).sum(axis=1)

# Step 4: Calculate RP_Exp
expense_fields = [
    "RPTS_Purchases",
    "RPTS_Interest Expenses",
    "RPTS_Director Remuneration",
    "RPTS_Rent Expense",
    "RPTS_Expenses",
    "RPTS_Dividend Expense",
    "RPTS_Discount Expense",
    "RPTS_Commission Expense",
    "RPTS_Claims Paid",
    "RPTS_Expenses Recovered",
    "RPTS_Diminution on Investments",
    "RPTS_Donations",
    "RPTS_Forex Loss",
    "RPTS_Loss on Sale of Assets",
    "RPTS_Loss on Sale of Investments",
    "RPTS_Provision for Doubtful Debts & Advances",
    "RPTS_Purchase of Assets",
    "RPTS_Purchase of Investments",
    "RPTS_Provision Write offs",
    "RPTS_Reimbursement Expense",
    "RPTS_Other Non Operating Expenses"
]

df_rp["RP_Exp"] = df_rp[expense_fields].fillna(0).sum(axis=1)

# Step 5: Drop used fields
df_rp = df_rp.drop(columns=income_fields + expense_fields + ["Sr.No."], errors="ignore")
df_rp.head()

,Accord Code,Company Name,RPTS_Year End,RP_Inc,RP_Exp
0,207635,20 Microns Ltd.,202403,10.78,10.32
1,207635,20 Microns Ltd.,202303,9.71,8.51
2,207635,20 Microns Ltd.,202203,7.44,7.71
3,224943,20 Microns Nano Minerals Ltd.,202403,4.24,10.14
4,224943,20 Microns Nano Minerals Ltd.,202303,3.87,10.67


In [5]:
import pandas as pd

# Step 1: Load CSV
df_fh = pd.read_csv(data_path / "FH Data.csv")

# Step 2: Drop rows where FH_Year End is blank
df_fh = df_fh.dropna(subset=["FH_Year End"])
df_fh = df_fh[df_fh["FH_Year End"].astype(str).str.strip() != ""]
df_fh = df_fh.reset_index(drop=True)

# Step 3: Drop Sr.No.
df_fh = df_fh.drop(columns=["Sr.No."], errors="ignore")
df_fh.head()

,Accord Code,Company Name,FH_Year End,FH_Net Worth
0,207635,20 Microns Ltd.,202403,323.28
1,207635,20 Microns Ltd.,202303,269.60
2,207635,20 Microns Ltd.,202203,233.53
3,224943,20 Microns Nano Minerals Ltd.,202403,44.30
4,224943,20 Microns Nano Minerals Ltd.,202303,41.02


In [11]:
# --- Step 1: Standardize Year_End column names ---
df_pl = df_pl.rename(columns={"PL_Year End": "Year_End"})
df_bs = df_bs.rename(columns={"BS_Year End": "Year_End"})
df_rp = df_rp.rename(columns={"RPTS_Year End": "Year_End"})
df_fh = df_fh.rename(columns={"FH_Year End": "Year_End"})

# --- Step 2: Drop duplicate 'Company Name' in other dfs ---
df_bs = df_bs.drop(columns=["Company Name"], errors="ignore")
df_rp = df_rp.drop(columns=["Company Name"], errors="ignore")
df_fh = df_fh.drop(columns=["Company Name"], errors="ignore")

# --- Step 3: Merge all dfs using df_pl as base ---
combined_df = df_pl.merge(df_bs, on=["Accord Code", "Year_End"], how="left", indicator="bs_flag")
combined_df = combined_df.merge(df_rp, on=["Accord Code", "Year_End"], how="left", indicator="rp_flag")
combined_df = combined_df.merge(df_fh, on=["Accord Code", "Year_End"], how="left", indicator="fh_flag")

# --- Step 4: Replace NaN with 0 for numeric fields ---
for col in combined_df.select_dtypes(include=["number"]).columns:
    combined_df[col] = combined_df[col].fillna(0)

# --- Step 5: Merge Audit ---
audit = {
    "Total PL rows": len(df_pl),
    "Matched with BS": (combined_df["bs_flag"] == "both").sum(),
    "Unmatched with BS": (combined_df["bs_flag"] == "left_only").sum(),
    "Matched with RP": (combined_df["rp_flag"] == "both").sum(),
    "Unmatched with RP": (combined_df["rp_flag"] == "left_only").sum(),
    "Matched with FH": (combined_df["fh_flag"] == "both").sum(),
    "Unmatched with FH": (combined_df["fh_flag"] == "left_only").sum(),
}

audit



{'Total PL rows': 47602,
 'Matched with BS': np.int64(45335),
 'Unmatched with BS': np.int64(2267),
 'Matched with RP': np.int64(37366),
 'Unmatched with RP': np.int64(10236),
 'Matched with FH': np.int64(47581),
 'Unmatched with FH': np.int64(21)}

In [12]:
import numpy as np

# Step 2: Calculate PLI
combined_df["PLI"] = (combined_df["OR"] - combined_df["OC"]) / combined_df["OC"]*100

# Step 3: Employee Cost %
combined_df["Emp_Cost_%"] = combined_df["PL_Employee Cost"].fillna(0) / combined_df["OC"]

# Step 4: Export Income %
combined_df["Export_Inc_%"] = combined_df["PL_      Sales - Exports"].fillna(0) / combined_df["OR"]

# Step 5: Service Income %
combined_df["Service_Inc_%"] = combined_df["PL_      Services Sales"].fillna(0) / combined_df["OR"]

# Step 6: RPT %
combined_df["RPT_%"] = np.maximum(
    combined_df["RP_Inc"].fillna(0) / combined_df["OR"],
    combined_df["RP_Exp"].fillna(0) / combined_df["OC"]
)

# Calculate new percentage columns
combined_df["Recv_%"] = (combined_df["BS_Sundry Debtors"] / combined_df["OR"]).fillna(0) * 100
combined_df["Intbls_%"] = (combined_df["Gross_Intbls"] / combined_df["OR"]).fillna(0) * 100

# Step 7: FY (convert Year_End like 202303 → "2022-23")
def year_to_fy(val):
    try:
        year = int(str(val)[:4])  # take YYYY from YYYYMM
        return f"{year-1}-{str(year)[-2:]}"
    except:
        return np.nan

combined_df["fy"] = combined_df["Year_End"].apply(year_to_fy)
# List of percentage columns
pct_cols = ["Emp_Cost_%", "Export_Inc_%", "Service_Inc_%", "RPT_%","Recv_%","Intbls_%"]

for col in pct_cols:
    combined_df[col] = (
        combined_df[col].fillna(0) * 100
    ).clip(lower=0, upper=100).round(2)

# Round PLI also to 2 decimals (but not clipped, since it can be negative or >100%)
combined_df["PLI"] = combined_df["PLI"].round(2)

print(combined_df.shape)
combined_df.head()

(47602, 29)


,Accord Code,Company Name,CD_Industry1,CD_CIN Number,Year_End,PL_ Sales - Exports,PL_ Manufacturing Sales,PL_ Trading Sales,PL_ Services Sales,PL_Employee Cost,...,FH_Net Worth,fh_flag,PLI,Emp_Cost_%,Export_Inc_%,Service_Inc_%,RPT_%,Recv_%,Intbls_%,fy
0,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202403,91.08,669.58,0.0,3.22,49.50,...,323.28,both,12.62,8.28,13.53,0.48,1.73,100.0,0.0,2023-24
1,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202303,86.34,595.42,0.0,2.78,44.13,...,269.60,both,10.78,8.16,14.42,0.46,1.62,100.0,0.0,2022-23
2,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,202203,82.75,514.78,0.0,2.79,36.68,...,233.53,both,11.89,7.93,15.98,0.54,1.67,100.0,0.0,2021-22
3,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,202403,8.51,92.14,0.0,0.00,8.23,...,44.30,both,9.88,9.80,9.23,0.00,12.08,100.0,0.0,2023-24
4,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,202303,9.12,87.57,0.0,0.49,9.45,...,41.02,both,8.94,11.67,10.34,0.56,13.18,100.0,0.0,2022-23


In [13]:

# Step 7: 
drop_cols = [
    "PL_      Sales - Exports",
    "PL_      Manufacturing Sales",
    "PL_      Trading Sales",
    "PL_      Services Sales",
    "PL_Employee Cost",
    "Year_End",
    "OC",
    "BS_Sundry Debtors",
    "Gross_Intbls",
    "bs_flag",
    "RP_Inc",
    "RP_Exp",
    "rp_flag",
    "fh_flag",
    "Recv_%"
]
combined_df = combined_df.drop(columns=drop_cols, errors="ignore")

# Step 8: 
# rename mapping
rename_dict = {
    "CD_CIN Number":"cin",
    "Company Name": "company_name",
    "CD_Industry1": "industry_sector",
    "Segment": "segment",
    "FH_Net Worth": "net_worth",
    "OR": "or",
    "PLI": "pli",
    "Emp_Cost_%": "emp_cost_pct",
    "RPT_%": "rpt_pct",
    "Export_Inc_%": "export_income_pct",
    "Service_Inc_%": "service_income_pct",
    "Intbls_%": "gross_intbl_pct"
}

# rename only existing cols
combined_df = combined_df.rename(columns={col: rename_dict[col] for col in combined_df.columns if col in rename_dict})


print(combined_df.shape)
combined_df.head()

(47602, 14)


,Accord Code,company_name,industry_sector,cin,or,segment,net_worth,pli,emp_cost_pct,export_income_pct,service_income_pct,rpt_pct,gross_intbl_pct,fy
0,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,673.17,Other,323.28,12.62,8.28,13.53,0.48,1.73,0.0,2023-24
1,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,598.95,Other,269.60,10.78,8.16,14.42,0.46,1.62,0.0,2022-23
2,207635,20 Microns Ltd.,Mining & Minerals,L99999GJ1987PLC009768,517.84,Other,233.53,11.89,7.93,15.98,0.54,1.67,0.0,2021-22
3,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,92.23,Other,44.30,9.88,9.80,9.23,0.00,12.08,0.0,2023-24
4,224943,20 Microns Nano Minerals Ltd.,Mining & Minerals,U15543GJ1993PLC020540,88.22,Other,41.02,8.94,11.67,10.34,0.56,13.18,0.0,2022-23


In [14]:
# Step 1: Drop rows where OR < 1
combined_df = combined_df[combined_df["or"] >= 1]

# Step 2: Drop rows where Segment is 'Other'
combined_df = combined_df[combined_df["segment"] != "Other"]

# Reset index after filtering
combined_df = combined_df.reset_index(drop=True)
combined_df.shape

(3110, 14)

In [15]:

# Step 1: Flag if OP is negative in each row
combined_df["pli_negative"] = combined_df["pli"] < 0

# Step 2: Group by company and check if all available OP values are negative
persistent_loss_map = (
    combined_df.groupby("Accord Code")["pli_negative"]
    .all()              # True if all are negative
    .astype(int)        # Convert True/False → 1/0
)

# Step 3: Map back to main dataframe
combined_df["persistent_loss"] = combined_df["Accord Code"].map(persistent_loss_map)

# Step 4: Drop helper column
combined_df = combined_df.drop(columns=["pli_negative","Accord Code"])
combined_df.shape


(3110, 14)

In [17]:

# Save the combined dataframe to CSV
out_path = BASE_DIR / "1. data" / "processed"
combined_df.to_csv(out_path / "ACETP_IT_Data_Processed.csv", index=False)
